In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")


D=6;
chi=10;
W=10;
N=20;
tol=1e-6;


Dtrun_init=200;
Dtrun_max=200;
trun_tol=1e-5;
group_size=Int(round((10^8)/(chi*chi*W*W*D)));


pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [2]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [3]:
O1=[];
O2=[];
Ag=[];
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now do CTMRG and itebd")


    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    @time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);
    
    @time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
    @time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");
    display((E_up+E_down)/3)

    
    Tleft=CTM["Tset"][4];
    Tright=CTM["Tset"][2];
    @tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
    @tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];
    
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"load itebd ground state from saved data directly"

In [4]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [5]:
# for s in sectors(space_AOA)
#   display(s)
#   @show s, dim(space_AOA, s), dim(s)
# end


In [6]:
check_complete_decomp=false;
if check_complete_decomp
    Dtrun=W*W*chi*chi;

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [7]:
euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
# display(euR_set)

eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
# display(eur_set)

S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
# display(S_set)

s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
# display(s_set)

"FLR_eig, DTrun=200, SU2 reduced to 86, minimal eigenvalue: 4.1467962877589396e-7"

"GLR_eig, Dtrun=200, SU2 reduced to 35, minimal eigenvalue: 5.525824144652159e-12"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"FLR_svd, DTrun=200, SU2 reduced to 84, minimal eigenvalue: 7.355582423099009e-7"

"GLR_svd, Dtrun=200, SU2 reduced to 35, minimal eigenvalue: 1.7974159912641293e-12"

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 11 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 201`); setting `howmany = 11`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 21 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested singular values (i.e. `howmany == 201`); setting `howmany = 21`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 13 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 101`); setting `howmany = 13`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 32 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the numb

(Any[[2.015002244747017, 0.047510382583433985, 0.02064923530410841, 0.005381295458955596], [0.33904165780312756, 0.33904165780312745, 0.0039785008693213305, 0.003978500869321326, 0.0001890946472343763, 0.00018909464723437162, 4.846427576803829e-5, 4.8464275768036555e-5], [0.006700713068213065, 0.004429618609512903, 0.0037865014547134868, 0.003128164179692512, 0.0007183850257752438, 0.0004986019062991714], [0.00023248096193525446, 0.00023248096100845, 4.0577790999993256e-5, 4.05777907727004e-5], Float64[]], Any[Any[TensorMap((Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1)') ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (false, true), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
[:, :, 1] =
   0.33257611776464274 + 3.031996647489913e-6im  …  -0.0019929406090944796 - 2.2195587358142067e-5im
 -0.001992941013464204 + 2.215924927168231e-5im        0.04838159937287393 + 4.410805204040051e-7im
* Data for fusiontree FusionTree{

In [55]:
euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_eig)

eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_eig)


S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_svd)

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_svd)


check_truncated_decomp_error=false;
if check_truncated_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(eur_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end

"group information:"

9-element Vector{Any}:
 UnitRange{Int64}[1:11]
 UnitRange{Int64}[1:20]
 UnitRange{Int64}[1:13]
 UnitRange{Int64}[1:4]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]
 ComplexF64[]

"group information:"

5-element Vector{Any}:
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:8]
 UnitRange{Int64}[1:6]
 UnitRange{Int64}[1:4]
 ComplexF64[]

"group information:"

...

"%%%%%%%%%%%%%%%%%%%%%%"

"check GLR_svd"

4.586685035791573e-6

4.586685035791574e-6

4.586685035791574e-6

In [9]:
function excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,VR,EU,VL,N,w,chi,d,k,DTrun_list)
    @assert pow==round((N-2)/2)
    DTrun=numpy.size(EU)
    x=torch.einsum('ab,b->a',input_transform,x)# do this calculation on cpu because the matrix 'input_transform' maybe large
    x=torch.as_tensor(x,device=Ag.device)
    x=torch.reshape(x,(w,w,d))
    
    H_eff_x=torch.zeros((w,w,d),dtype=Ag.dtype,device=Ag.device)
    cm=0
    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            H_eff_x0=torch.zeros((w,w,d),dtype=Ag.dtype,device=Ag.device)
            for Dset in DTrun_list:
                H_eff_x0=H_eff_x0+coe*contract_H_eff_trun(x,A1,A2,Ag,torch.as_tensor(VR[:,Dset],device=Ag.device),torch.as_tensor(EU[Dset],device=Ag.device),torch.as_tensor(VL[Dset,:],device=Ag.device),w,chi,d,cm,cn,pow,N,Dset)
            end
            H_eff_x=H_eff_x+H_eff_x0
    end
    x=torch.reshape(H_eff_x,(-1,)).cpu()
    x_output=torch.einsum('ab,b->a',output_transform,x)
    return x_output
end

function excitation_TrunTransOp_iterative_norm_eff(x,input_transform,output_transform,A1,A2,Ag,pow,vr,eu,vl,N,w,chi,d,k,object,DTrun_list)
    @assert pow==round((N-2)/2)
    norm_eff=torch.zeros((w,w,d,w,w,d),dtype=Ag.dtype,device=Ag.device)
    cm=0
    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            norm_eff=norm_eff+coe*contract_norm_eff(Ag,cm,cn,N)
    end
    return norm_eff
end


-2.718281828459045 + 3.328935140402784e-16im

In [ ]:
function v_Transop(VL_comp,Ag,A1,A2,L)
    VL_comp=copy(VL_comp)
    for cl=1:L
        @tensor VL_comp[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
    end
    return VL_comp
end


function contract_H_eff_trun(x,A1,A2,Ag,VR_comp,EU_comp,VL_comp,w,chi,d,cm,cn,pow,N,Dset)
    H_eff_x=zeros(Complex{Float64},(w,w,d))
    L_Ea=cn-cm-1
    L_Eb=N-1-cn
    if L_Ea>=L_Eb
        L=L_Ea
        decomp_posit="Ea"
    elseif L_Ea<L_Eb
        L=L_Eb
        decomp_posit="Eb"
    end
    Dsiz=numpy.size(Dset)


    VL_comp=v_Transop(VL_comp,Ag,A1,A2,L-pow);
    VR_comp=torch.permute(torch.reshape(torch.einsum('ab,bc->ac',VR_comp,torch.complex(torch.diag(EU_comp),torch.diag(EU_comp)*0)),(w,chi,chi,w,Dsiz)),(0,2,1,3,4))
    if cm==cn: # cm=cn=0
        @tensor ttt[:]:=VL_comp[9,-1,5,3,1]*A2[5,4,6,-3]*A1[3,2,7,4]*x[1,8,2]*VR_comp[-2,6,7,8,9];
        H_eff_x=H_eff_x+ttt;
    elseif cn==2
        @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
        @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,-5,5,4]*VR_comp[1,3,5,-4,-6];
        @tensor T1T2x[:]:=T1[6,-1,3,4,5,-3]*T2[-2,3,4,2,1,6]*x[5,2,1];
        H_eff_x=H_eff_x+T1T2x;
    elseif cn==N
        @tensor T1[:]:=VL_comp[-1,1,3,5,-2]*Ag'[1,-3,2]*A2[3,4,-4,2]*A1[5,-6,-5,4];
        @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
        @tensor T1xT2[:]:=T1[6,1,-1,3,4,2]*x[1,5,2]*T2[3,4,5,-2,-3,6];
        H_eff_x=H_eff_x+T1xT2;
    else
        if  decomp_posit=="Ea"
            @tensor T1[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*x,[7,-5,6];
            @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
            @assert L_Eb>=1
            for cr in range(1,L_Eb+1):
                @tensor T1[:]:=T1[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3]*T2[1,2,3,-2,-3,4];
            H_eff_x=H_eff_x+T1T2;
        elseif decomp_posit=="Eb"
            @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
            @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*x[-4,7,6]*VR_comp[1,3,5,7,-5];
            @assert L_Ea>=1
            for cr in range(1,L_Ea+1)
                @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*Ag[-4,7,6]*T2[1,3,5,7,-5];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3,-3]*T2[-2,1,2,3,4];
            H_eff_x=H_eff_x+T1T2;
        end
    end
    return H_eff_x
end


    

In [45]:
function contract_norm_eff(AL,cm,cn,N)

    mps=create_puMPS(AL,N);
    mps1=mps;
    mps2=mps;
    if cm==cn # cm=cn=1
        E=contiguous_TM(mps1,mps2,2:N)
        Id=id(space(Ag,3));
        @tensor norm_eff[:]:=E[-2,-5,-1,-4]*Id[-3,-6];
    elseif cn==2
        E=contiguous_TM(mps1,mps2,3:N);
        @tensor norm_eff[:]:=mps2[1][1,-4,-3]*mps1[2]'[-2,2,-6]*E[2,-5,-1,1];
    elseif cn==N
        E=contiguous_TM(mps1,mps2,2:N-1);
        @tensor norm_eff[:]:=mps2[1][-5,1,-3]*E[-2,1,2,-4]*mps1[N]'[2,-1,-6]
    else
        T1=mps2[1];
        T2=contiguous_TM(mps1,mps2,2:(cn-1))
        T3=mps1[cn]';
        T4=contiguous_TM(mps1,mps2,(cn+1):N)
        @tensor norm_eff[:]:=T1[6,1,-3]*T2[-2,1,3,-4]*T3[3,4,-6]*T4[4,-5,-1,6];
    end
    return norm_eff
end



contract_norm_eff (generic function with 1 method)

In [27]:
function contiguous_TM(mps1,mps2,positions)
    if length(positions)==1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
    elseif length(positions)>1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
        for ci=positions[2]:positions[end]
            @tensor E[:]:=E[-1,-2,1,2]*mps1[ci]'[1,-3,3]*mps2[ci][2,-4,3];
        end
    end
    return E
end

contiguous_TM (generic function with 1 method)

In [19]:

function create_puMPS(A,N)
    mps=Vector{Any}(undef, N);
    for c1=1:N
        mps[c1]=A
    end
    return mps
end

create_puMPS (generic function with 1 method)

In [31]:
display(space(Ag,3))
display(id(space(Ag,3)))


Rep[SU₂](0=>1, 1/2=>1, 1=>1)'

TensorMap(ProductSpace(Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace(Rep[SU₂](0=>1, 1/2=>1, 1=>1)')):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (true,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (true,), ()):
 1.0
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2,), 1/2, (true,), ()) ← FusionTree{Irrep[SU₂]}((1/2,), 1/2, (true,), ()):
 1.0
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (true,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (true,), ()):
 1.0


In [44]:
norm_eff=contract_norm_eff(Ag,1,1,N);
display(space(norm_eff))


(Rep[SU₂](0=>2, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)' ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}() ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [52]:
display(S_set_co)


UndefVarError: UndefVarError: S_set_combined not defined

In [ ]:
function solve_ITEBD_excitation_TrunTransOp_iterative(Ag,A1,A2,n_E,N,kset,pow,evr,lambdar,evr_inv,evR,lambdaR,evR_inv,u,s,vt,U,S,Vt,Dtrun_method,DTrun_group)
    
    #w=
    #chi=
    #d=
    
    Eset=Vector{Any}(undef, length(kset)); 
    DTrun=numpy.size(S)
    display("DTrun="*str(DTrun))

    DTrun_list=[]
    inds=numpy.linspace(0, DTrun-1, num=DTrun, dtype=int)
    if DTrun_group:
        group_size=round((10**8)/(chi*chi*w*w*d))
    else:
        group_size=1
    if group_size<=DTrun:
        for cc in range(0,int((DTrun-DTrun%group_size)/group_size)):
            DTrun_list.append(inds[list(range(cc*group_size,(cc+1)*group_size))])
        if DTrun%group_size>0:
            DTrun_list.append(inds[list(range(-1-int(DTrun%group_size)+1,0))])
    else:
        DTrun_list.append(inds)
    print(DTrun_list)


    for kk in range(0,numpy.size(kset)):
        ck=kset[kk]
        print(Ag.device)
        start_time = time.time()
        k=2*math.pi*ck/N
        print('sector '+'k='+str(ck)+':')


        norm_eff=excitation_TrunTransOp_iterative_norm_eff([],[],[],A1,A2,Ag,pow,[],[],[],N,w,chi,d,k,DTrun_list).cpu() # put it on cpu because this matrix maybe large
        norm_eff=torch.reshape(norm_eff,(w*w*d,w*w*d))
        uu,ss,vvt=torch.linalg.svd(norm_eff)
        norm_eff=[]#clear this big matrix

        position=(numpy.abs(ss.cpu().numpy())/numpy.max(numpy.abs(ss.cpu().numpy()))>1e-10).nonzero()
        position=numpy.reshape(position,-1)
        ss=ss[position]
        input_transform=torch.permute(torch.conj(vvt[position,:]),(1,0))
        output_transform=torch.permute(torch.conj(uu),(1,0))
        output_transform=output_transform[position,:]
        output_transform=torch.mm(torch.diag(torch.complex(torch.div(1,ss),torch.div(1,ss)*0)),output_transform)
        
        
        if Dtrun_method=='svds':
            # u=torch.as_tensor(u,device=Ag.device)
            # s=torch.as_tensor(s,device=Ag.device)
            # vt=torch.as_tensor(vt,device=Ag.device)
            # U=torch.as_tensor(U,device=Ag.device)
            # S=torch.as_tensor(S,device=Ag.device)
            # Vt=torch.as_tensor(Vt,device=Ag.device)
            H=lambda x : excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,U,S,Vt,N,w,chi,d,k,DTrun_list)
            op = spla.LinearOperator(
                matvec=H,
                dtype=Ag.cpu().numpy().dtype,
                shape=(numpy.size(position),numpy.size(position))
            )
        elif Dtrun_method=='eigs':
            # evr=torch.as_tensor(evr,device=Ag.device)
            # lambdar=torch.as_tensor(lambdar,device=Ag.device)
            # evr_inv=torch.as_tensor(evr_inv,device=Ag.device)
            # evR=torch.as_tensor(evR,device=Ag.device)
            # lambdaR=torch.as_tensor(lambdaR,device=Ag.device)
            # evR_inv=torch.as_tensor(evR_inv,device=Ag.device)
            H=lambda x : excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,evR,lambdaR,evR_inv,N,w,chi,d,k,DTrun_list)
            op = spla.LinearOperator(
                matvec=H,
                dtype=Ag.cpu().numpy().dtype,
                shape=(numpy.size(position),numpy.size(position))
            )

        Es,_=spla.eigs(op,n_E,which="LM");
        Eset[kk]=Es
        print(Es)
        print("--- %s seconds ---" % (time.time() - start_time))
    return kset, Eset
end
